In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sns
import mlflow
import dagshub
import json
import os
import keras
import joblib
import matplotlib.pyplot as plt
from dagshub import dagshub_logger
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, Callback
from sklearn.metrics import confusion_matrix, classification_report
from kerastuner.tuners import RandomSearch

2025-05-22 16:31:38.017026: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-22 16:31:38.044387: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747924298.067634   78376 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747924298.074405   78376 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747924298.099186   78376 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
df = pd.read_csv('../data/landmarked/landmarked_dataset.csv')
df.head()

,letter,landmark_0_x,landmark_0_y,landmark_0_z,landmark_1_x,landmark_1_y,landmark_1_z,landmark_2_x,landmark_2_y,landmark_2_z,...,landmark_17_z,landmark_18_x,landmark_18_y,landmark_18_z,landmark_19_x,landmark_19_y,landmark_19_z,landmark_20_x,landmark_20_y,landmark_20_z
0,t,0.534516,0.774727,-1.003871e-06,0.508589,0.632008,-0.015615,0.521696,0.490609,-0.034787,...,-0.080039,0.415002,0.606962,-0.091086,0.423466,0.632120,-0.082430,0.461077,0.646738,-0.078768
1,t,0.401396,0.673921,-3.702490e-07,0.452425,0.515237,0.035717,0.514422,0.376457,-0.008155,...,-0.266454,0.583444,0.575833,-0.254681,0.550737,0.601097,-0.220273,0.512921,0.593272,-0.211116
2,t,0.516662,0.857623,-9.007788e-07,0.467261,0.676344,-0.032283,0.385492,0.530903,-0.070112,...,-0.129384,0.305695,0.878271,-0.155441,0.351569,0.876547,-0.137212,0.419136,0.865122,-0.127641
3,t,0.425197,0.716900,-2.199591e-07,0.503539,0.551109,0.007586,0.546467,0.395048,-0.014543,...,-0.152825,0.531544,0.609670,-0.170653,0.531558,0.631256,-0.153000,0.493700,0.630563,-0.141868
4,t,0.589113,0.753619,-8.772182e-07,0.535902,0.604019,-0.033694,0.506365,0.464442,-0.058517,...,-0.090931,0.448059,0.649945,-0.110480,0.462349,0.663688,-0.099754,0.501952,0.661236,-0.091881


In [3]:
label_encoder = LabelEncoder()
scaler = StandardScaler()

classes = sorted(df['letter'].unique())

label_encoder.fit(classes)

y = df['letter']
y = label_encoder.transform(y)

X = df.drop('letter', axis=1)

# 70% train, 15% val, 15% test
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=(15/85), random_state=42, stratify=y_temp)

print(f"Partitions shape:\nTrain{X_train.shape}\nValidation: {X_val.shape}\nTest:{X_test.shape}")

X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

Partitions shape:
Train(8843, 63)
Validation: (1895, 63)
Test:(1896, 63)


In [4]:
joblib.dump(scaler, 'scaler.pkl')
joblib.dump(label_encoder, 'label_encoder.pkl')

['label_encoder.pkl']

In [5]:
# Defining mlflow experiment parameter
TRIAL_NAME = "v2_augmented_trial_"
MLFLOW_MAIN_RUN = "Main Tunining Run: Landmark Model"
TUNER_DIRECTORY = "logs_lm_model/v2_augmented_tuner"
TUNER_PROJECT_NAME = "FingerSpellIT - v2_Augmented Landmark Best Model"
MODEL_NAME = "model_landmarked_v3"
EVALUATION_MLFLOW_RUN = "Evaluating v2_Augmented Landmark Best Model"
REPORT_NAME = "v2_augmented_classification_report"
CM_NAME="v2_augmented_confusion_matrix"

In [6]:
# MLflow - Dagshub initialization
mlflow.set_tracking_uri("https://dagshub.com/alfoCaiazza/FingerSpellIT.mlflow")

dagshub.init(repo_owner='alfoCaiazza', repo_name='FingerSpellIT', mlflow=True)
dagshub_log = dagshub_logger(metrics_path="metrics", hparams_path="params")

Accessing as alfoCaiazza

Initialized MLflow to track repo "alfoCaiazza/FingerSpellIT"

Repository alfoCaiazza/FingerSpellIT initialized!

In [7]:
# To avoid OOM errors, setting GPU Memory Consuption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    print(f"GPU: {gpu}")
    tf.config.experimental.set_memory_growth(gpu, True) # Keeping the use of memory limited to prevent errors

GPU: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [8]:
def build_model(hp):
    # Input Layer
    input_layer = layers.Input(shape=(63,))
    x = input_layer
    
    # First Dense Layer
    activation = hp.Choice('initial_activation', ['relu', 'tanh', 'sigmoid'])
    x = layers.Dense(64, activation=activation)(x)
    x = layers.Dropout(hp.Float('initial_dropout', 0.1, 0.5, step=0.1))(x)
    
    # Tunable Hidden Layers
    for i in range(hp.Int('num_layers', 1, 4)):
        units = hp.Int(f'units_{i}', 128, 512, step=128)
        x = layers.Dense(units, activation=activation)(x)

        # Optional BatchNorm
        if hp.Boolean(f'use_batchnorm_{i}'):
            x = layers.BatchNormalization()(x)
        x = layers.Activation(activation)(x)
            
        x = layers.Dropout(hp.Float(f'dropout_{i}', 0.1, 0.5, step=0.1))(x)
    
    # Output Layer
    prediction = layers.Dense(24, activation='softmax')(x)
    
    # Compile Model
    optimizer_name = hp.Choice('optimizer', ['adam', 'rmsprop', 'sgd'])
    
    if optimizer_name == 'adam':
        optimizer = keras.optimizers.Adam(
            learning_rate=hp.Float('adam_lr', 1e-5, 1e-2, sampling='log')
        )
    elif optimizer_name == 'rmsprop':
        optimizer = keras.optimizers.RMSprop(
            learning_rate=hp.Float('rmsprop_lr', 1e-5, 1e-2, sampling='log'),
            rho=hp.Float('rmsprop_rho', 0.8, 0.99)
        )
    elif optimizer_name == 'sgd':
        optimizer = keras.optimizers.SGD(
            learning_rate=hp.Float('sgd_lr', 1e-4, 1e-1, sampling='log'),
            momentum=hp.Float('sgd_momentum', 0.0, 0.99)
        )
    
    model = Model(inputs=input_layer, outputs=prediction)
    model.compile(
        optimizer=optimizer,
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy', 'sparse_categorical_accuracy']
    )
    
    return model

In [9]:
base_callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', patience=2, factor=0.5, min_lr=1e-5)
]

# MLflow tracking callback
class MLflowCallback(Callback):
    def __init__(self, trial_hyperparameters, trial_id):
        super().__init__()
        self.trial_id = trial_id
        self.trial_hyperparameters = trial_hyperparameters

    def on_train_begin(self, logs=None):
        self.run =  mlflow.start_run(run_name=f"{TRIAL_NAME}_{self.trial_id}", nested=True)

        mlflow.log_param('trail_id', self.trial_id)
        for param_name, param_value in self.trial_hyperparameters.values.items():
            mlflow.log_param(param_name, param_value)


    def on_epoch_end(self, epoch, logs=None):
        if logs is not None:
            for metric_name, value in logs.items():
                mlflow.log_metric(metric_name, value, step=epoch)

    def on_train_end(self, logs=None):
        if self.run:
            mlflow.end_run()

In [10]:
# Subclassed RandomSerach tuner which uses customized MLflow callback
class MLflowTuner(RandomSearch):
    def run_trial(self, trial, *args, **kwargs):
        callbacks = base_callbacks + [MLflowCallback(trial.hyperparameters, trial.trial_id)]
        kwargs['callbacks'] = callbacks
        return super().run_trial(trial, *args, **kwargs)

In [11]:
# Initializing the Tuner
with mlflow.start_run(run_name=f"{MLFLOW_MAIN_RUN}"):
    epochs = 50
    mlflow.log_param('epochs', epochs)

    tuner = MLflowTuner(
        build_model,
        objective='sparse_categorical_accuracy',
        max_trials=10, 
        executions_per_trial=1,
        directory=f"{TUNER_DIRECTORY}",
        project_name=f"{TUNER_PROJECT_NAME}"
    )

    tuner.search(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs
    )

    best_model = tuner.get_best_models(num_models=1)[0]
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
    best_trial = tuner.oracle.get_best_trials(num_trials=1)[0]

    for param, value in best_hps.values.items():
        mlflow.log_param(param, value)

    for metric, value in best_trial.metrics.metrics.items():
        if metric and isinstance(metric, dict):
            values = value.get('value', [])
            if values:
                mlflow.log_metric(metric, values[-1])

    model_path = f"../model/{MODEL_NAME}.h5"
    best_model.save(model_path)
    mlflow.log_artifact(model_path)

mlflow.end_run()

Trial 10 Complete [00h 05m 53s]
sparse_categorical_accuracy: 0.9566888809204102

Best sparse_categorical_accuracy So Far: 0.9566888809204102
Total elapsed time: 01h 01m 32s


/home/acaia/LIS/LIS-Gesture-Translator/.lis/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 34 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


🏃 View run Main Tunining Run: Landmark Model at: https://dagshub.com/alfoCaiazza/FingerSpellIT.mlflow/#/experiments/0/runs/a9f3b0dc36764249ad00e6f7b559ee87
🧪 View experiment at: https://dagshub.com/alfoCaiazza/FingerSpellIT.mlflow/#/experiments/0


In [12]:
results = best_model.evaluate(X_test, y_test)
print("Evaluation results:", results)

y_pred_probs = best_model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)

y_true = np.array(y_test)

48/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9719 - loss: 0.0747 - sparse_categorical_accuracy: 0.9719

2025-05-22 17:33:48.239494: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_78', 112 bytes spill stores, 112 bytes spill loads

2025-05-22 17:33:48.927800: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_78', 220 bytes spill stores, 220 bytes spill loads

2025-05-22 17:33:50.417357: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_100', 76 bytes spill stores, 76 bytes spill loads

2025-05-22 17:33:50.498930: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_84', 60 bytes spill stores, 60 bytes spill loads



60/60 ━━━━━━━━━━━━━━━━━━━━ 7s 92ms/step - accuracy: 0.9733 - loss: 0.0727 - sparse_categorical_accuracy: 0.9733
Evaluation results: [0.061949390918016434, 0.9799578189849854, 0.9799578189849854]
60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step


In [13]:
with mlflow.start_run(run_name=f"{EVALUATION_MLFLOW_RUN}"):
    class_names = label_encoder.classes_.tolist()
    report_path =f'../model/artifacts/{REPORT_NAME}.json'
    report = classification_report(y_true, y_pred, target_names=class_names, output_dict=True)
    print("Classification Report", json.dumps(report, indent=4))

    with open(report_path, "w") as f:
        json.dump(report, f, indent=4)

    mlflow.log_artifact(report_path, "evaluation_metrics")

    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.tight_layout()

    cm_local_path = os.path.join("../model/artifacts", f"{CM_NAME}.png")
    plt.savefig(cm_local_path, dpi=300, bbox_inches='tight')
    
    mlflow.log_figure(plt.gcf(), f"evaluation_plots/{CM_NAME}.png")

    plt.close()


Classification Report {
    "a": {
        "precision": 0.9733333333333334,
        "recall": 1.0,
        "f1-score": 0.9864864864864865,
        "support": 73.0
    },
    "b": {
        "precision": 0.82,
        "recall": 0.9318181818181818,
        "f1-score": 0.8723404255319149,
        "support": 44.0
    },
    "c": {
        "precision": 1.0,
        "recall": 1.0,
        "f1-score": 1.0,
        "support": 71.0
    },
    "d": {
        "precision": 1.0,
        "recall": 0.9285714285714286,
        "f1-score": 0.9629629629629629,
        "support": 56.0
    },
    "e": {
        "precision": 1.0,
        "recall": 1.0,
        "f1-score": 1.0,
        "support": 85.0
    },
    "f": {
        "precision": 0.9436619718309859,
        "recall": 0.9305555555555556,
        "f1-score": 0.9370629370629371,
        "support": 72.0
    },
    "g": {
        "precision": 1.0,
        "recall": 1.0,
        "f1-score": 1.0,
        "support": 120.0
    },
    "h": {
        "precisi